<a href="https://colab.research.google.com/github/MrBigBrane/Machine-Learning/blob/main/Kaggle_Predicting_House_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [52]:
train_data = pd.read_csv("/content/train.csv")
test_data = pd.read_csv("/content/test.csv")

In [53]:
# Drop the label column and get label set
X_train_data = train_data.drop("SalePrice", axis=1)
y_train_data = train_data["SalePrice"]

X_test_data = test_data.drop(columns='Id')
X_test_copy = test_data.copy()

# Id is not useful for prediction
X_train_data.drop(columns='Id',inplace=True)
X_test_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [54]:
# Dropping any column that has more than 50% n/a values
for col in X_train_data.columns:
    if (X_train_data[col].isnull().sum()/X_train_data.shape[0])*100>50:
        X_train_data.drop(columns=col,inplace=True)

for col in X_test_data.columns:
    if (X_test_data[col].isnull().sum()/X_test_data.shape[0])*100>50:
        X_test_data.drop(columns=col,inplace=True)

In [55]:
# Get column names that have string values for onehotencoding
# string_cols = X_train_data.select_dtypes(include=['object']).columns
string_cols_test = X_test_data.select_dtypes(include=['object']).columns

# Get column names that have numerical data
# num_cols = X_train_data.select_dtypes(include=['int64', 'float64']).columns
num_cols_test = X_test_data.select_dtypes(include=['int64', 'float64']).columns

In [56]:
# Filling n/a values in columns with numerical data with median of each column
X_train_data[num_cols_test] = X_train_data[num_cols_test].fillna(X_train_data[num_cols_test].median())
X_test_data[num_cols_test] = X_test_data[num_cols_test].fillna(X_test_data[num_cols_test].median())

# Filling n/a values in columns with letter/string data with the most common value of each column
X_train_data[string_cols_test] = X_train_data[string_cols_test].fillna(X_train_data[string_cols_test].mode().iloc[0])
X_test_data[string_cols_test] = X_test_data[string_cols_test].fillna(X_test_data[string_cols_test].mode().iloc[0])

In [57]:
# Check that there are no N/A values left
for col in X_train_data.columns:
    if X_train_data[col].isnull().sum()>0:
        print(col,'->',X_train_data[col].value_counts())

for col in X_test_data.columns:
    if X_test_data[col].isnull().sum()>0:
        print(col,'->',X_test_data[col].value_counts())

FireplaceQu -> FireplaceQu
Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: count, dtype: int64


In [58]:
X_train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal


In [59]:
X_train_data.nunique()

,0
MSSubClass,15
MSZoning,5
LotFrontage,110
LotArea,1073
Street,2
...,...
MiscVal,21
MoSold,12
YrSold,5
SaleType,9


### One Hot Encoding

In [60]:
# One hot encode only the string value columns
X_train_encoded = pd.get_dummies(X_train_data, columns=string_cols_test, drop_first=True)
X_test_encoded = pd.get_dummies(X_test_data, columns=string_cols_test, drop_first=True)

# Columns with non-binary values (to distinguish for scaling)
# non_binary_columns = [
#     col for col in X_train_encoded.columns if X_train_encoded[col].nunique() != 2
# ]
# # Columns with only binary values (to distinguish for scaling)
# binary_columns = [
#     col for col in X_train_encoded.columns if X_train_encoded[col].nunique() == 2
# ]

non_binary_columns_test = [
    col for col in X_test_encoded.columns if X_test_encoded[col].nunique() != 2
]
binary_columns_test = [
    col for col in X_test_encoded.columns if X_test_encoded[col].nunique() == 2
]

# Cast boolean ohe to int
X_train_encoded[binary_columns_test] = X_train_encoded[binary_columns_test].astype(int)
X_test_encoded[binary_columns_test] = X_test_encoded[binary_columns_test].astype(int)

In [61]:
common_string_cols = list(set(X_train_encoded) & set(X_test_encoded.columns))
X_train_encoded = X_train_encoded[common_string_cols]
X_test_encoded = X_test_encoded[common_string_cols]

common_string_cols_catboost = list(set(X_train_data) & set(X_test_data.columns))
X_train_data = X_train_data[common_string_cols_catboost]
X_test_data = X_test_data[common_string_cols_catboost]

### Split and Scale Data

In [63]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X_train_encoded, y_train_data, test_size=0.2)


# Scale non-binary data columns with min max scaler (worked best)
scaler = MinMaxScaler()
X_train[non_binary_columns_test] = scaler.fit_transform(X_train[non_binary_columns_test])
X_test[non_binary_columns_test] = scaler.transform(X_test[non_binary_columns_test])

# Scale catboost data
X_train_data[num_cols_test] = scaler.fit_transform(X_train_data[num_cols_test])
X_test_data[num_cols_test] = scaler.transform(X_test_data[num_cols_test])

# Scale test data
X_test_encoded[non_binary_columns_test] = scaler.transform(X_test_encoded[non_binary_columns_test])

## CatBoost

In [64]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.2 MB/s eta 0:00:00


In [73]:
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter distribution
param_dist = {
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'iterations': [100, 300, 500],
    'l2_leaf_reg': [1, 3, 5, 10],
    'bagging_temperature': [0, 1, 2, 5, 10]
}

model = CatBoostRegressor(loss_function='RMSE', verbose=0)

# Random search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, scoring='neg_mean_squared_error', random_state=42)
random_search.fit(X_train, y_train)

# Best parameters
print("Best parameters found:", random_search.best_params_)

best_params = random_search.best_params_

Best parameters found: {'learning_rate': 0.1, 'l2_leaf_reg': 3, 'iterations': 500, 'depth': 6, 'bagging_temperature': 0}


In [74]:
final_model = CatBoostRegressor(
    depth=best_params['depth'],
    learning_rate=best_params['learning_rate'],
    iterations=best_params['iterations'],
    l2_leaf_reg=best_params['l2_leaf_reg'],
    bagging_temperature=best_params['bagging_temperature'],
    loss_function='RMSE',
    verbose=0
)

# Train the model
final_model.fit(X_train, y_train)

In [75]:
from sklearn import metrics

#### Test dataset - metrics ####
y_test_pred = final_model.predict(X_test)
r2_score = round(metrics.r2_score(y_test, y_test_pred),2)
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)),2)
print(f'r2: {r2_score}')
print(f'rmse: {rmse}')

r2: 0.81
rmse: 29700.91


## Testing Lasso Model

In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

# Define parameter grid
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100, 150, 200, 300, 500]}

# Grid search for Lasso
lasso = Lasso(max_iter=10000)
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best alpha
print("Best alpha:", grid_search.best_params_['alpha'])

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.406e+10, tolerance: 6.394e+08
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.701e+11, tolerance: 6.376e+08
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.724e+11, tolerance: 6.027e

Best alpha: 200


In [70]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'alpha': 200}


In [71]:
final_model = Lasso(
    alpha=best_params['alpha'],
    max_iter=10000
)

# Train the model
final_model.fit(X_train, y_train)

Lasso(alpha=200, max_iter=10000)

In [72]:
from sklearn import metrics

#### Test dataset - metrics ####
y_test_pred = final_model.predict(X_test)
r2_score = round(metrics.r2_score(y_test, y_test_pred),2)
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)),2)
print(f'r2: {r2_score}')
print(f'rmse: {rmse}')

r2: 0.81
rmse: 30225.51


In [40]:
submission_pred = final_model.predict(X_test_encoded)

print(submission_pred)

[100663.89521516 153274.22493236 168536.87950673 ... 165429.71659969
 107744.31685238 231186.66679263]


In [41]:
ans=pd.DataFrame({'Id':X_test_copy['Id'],'SalePrice':submission_pred})
ans.to_csv('submission.csv',index=False)

## ElasticNet

In [42]:
from sklearn.linear_model import ElasticNet

# Define the parameter grid
param_grid = {
    'alpha': [0.01, 0.1, 1, 10],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9]
}

# ElasticNet model
elastic_net = ElasticNet(max_iter=10000)

# Grid search
grid_search = GridSearchCV(elastic_net, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters:", grid_search.best_params_)

best_params = grid_search.best_params_

Best parameters: {'alpha': 0.01, 'l1_ratio': 0.5}


In [43]:
final_model = ElasticNet(
    alpha=best_params['alpha'],
    l1_ratio=best_params['l1_ratio'],
    max_iter=10000
)

# Train the model
final_model.fit(X_train, y_train)

ElasticNet(alpha=0.01, max_iter=10000)

In [44]:
from sklearn import metrics

#### Test dataset - metrics ####
y_test_pred = final_model.predict(X_test)
r2_score = round(metrics.r2_score(y_test, y_test_pred),2)
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)),2)
print(f'r2: {r2_score}')
print(f'rmse: {rmse}')

r2: 0.85
rmse: 31434.99


## Ridge

In [45]:
from sklearn.linear_model import Ridge

# Define the parameter grid
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}

# Ridge model
ridge = Ridge()

# Grid search
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters:", grid_search.best_params_)

best_params = grid_search.best_params_

Best parameters: {'alpha': 10}


In [46]:
final_model = Ridge(
    alpha=best_params['alpha'],
)

# Train the model
final_model.fit(X_train, y_train)

Ridge(alpha=10)

In [47]:
#### Test dataset - metrics ####
y_test_pred = final_model.predict(X_test)
r2_score = round(metrics.r2_score(y_test, y_test_pred),2)
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)),2)
print(f'r2: {r2_score}')
print(f'rmse: {rmse}')

r2: 0.85
rmse: 32196.04


## Random Forest

In [48]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

# Define the RandomForestRegressor model
rf = RandomForestRegressor(random_state=42)

# Define the parameter distribution
param_dist = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Set up RandomizedSearchCV with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                   n_iter=25, cv=5, scoring='neg_mean_squared_error',
                                   n_jobs=-1, verbose=2, random_state=42)

# Fit the random search to the data
random_search.fit(X_train, y_train)

# Get the best parameters and model
print("Best parameters found: ", random_search.best_params_)
best_model_random = random_search.best_estimator_


Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best parameters found:  {'n_estimators': 500, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}


In [49]:
best_params = random_search.best_params_

final_model = RandomForestRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    max_features=best_params['max_features'],
    bootstrap=best_params['bootstrap'],
    random_state=42
)

# Train the model
final_model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',
                      min_samples_split=5, n_estimators=500, random_state=42)

In [50]:
#### Test dataset - metrics ####
y_test_pred = final_model.predict(X_test)
r2_score = round(metrics.r2_score(y_test, y_test_pred),2)
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)),2)
print(f'r2: {r2_score}')
print(f'rmse: {rmse}')

r2: 0.86
rmse: 30817.66


## XGBoost

In [21]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.3.1

Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 84.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.3.1 which is incompatible.


In [77]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid for random search
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9, 11],
    'min_child_weight': [1, 3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'alpha': [0, 0.1, 0.2],
    'lambda': [0, 0.1, 0.2]
}

# Initialize the XGBoost regressor
xg_reg = xgb.XGBRegressor(objective='reg:squarederror')

# Use RandomizedSearchCV to search for best parameters
random_search = RandomizedSearchCV(estimator=xg_reg, param_distributions=param_dist,
                                   n_iter=25, cv=5, scoring='neg_mean_squared_error',
                                   n_jobs=-1, verbose=2, random_state=42)

# Fit the model
random_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best parameters: {'subsample': 0.8, 'n_estimators': 400, 'min_child_weight': 3, 'max_depth': 11, 'learning_rate': 0.01, 'lambda': 0.2, 'gamma': 0.1, 'colsample_bytree': 0.6, 'alpha': 0}
Best score: -755946135.4099842


In [78]:
best_params = random_search.best_params_

final_model = xgb.XGBRegressor(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    min_child_weight=best_params['min_child_weight'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    gamma=best_params['gamma'],
    alpha=best_params['alpha'],
    lambda_=best_params['lambda'],
    objective='reg:squarederror'
)

# Train the model
final_model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:36:09] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "lambda_" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRegressor(alpha=0, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda_=0.2, learning_rate=0.01,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=11, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=400, n_jobs=None, ...)

In [79]:
from sklearn import metrics

#### Test dataset - metrics ####
y_test_pred = final_model.predict(X_test)
r2_score = round(metrics.r2_score(y_test, y_test_pred),2)
rmse = round(np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)),2)
print(f'r2: {r2_score}')
print(f'rmse: {rmse}')

r2: 0.8
rmse: 30985.56
